In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from utils import modeling 

In [2]:
ground_truth = [0,2]
print(modeling.accuracy(ground_truth,modeling.decision(modeling.sample(0,1,2),0.5)))
print(modeling.accuracy(ground_truth,modeling.decision(modeling.sample(1,1,2),0.5)))

[0]
[0, 0]
([0, 0], 0.0)
[0]
[0, 0]
([0, 0], 0.0)


In [3]:
ground_truth = [2,1]
print(modeling.accuracy(ground_truth,modeling.decision(modeling.sample(0,1,2),0.4)))
print(modeling.accuracy(ground_truth,modeling.decision(modeling.sample(0.5,1,2),0.4)))

[0]
[0, 0]
([0, 0], 0.0)
[0]
[0, 0]
([0, 0], 0.0)
